# **Import**

In [ ]:
"""
Import required libraries
"""
!pip install rasterio


# Raster processing
import rasterio

# Data handling
import numpy as np
import pandas as pd

# Deep learning (TensorFlow and Keras)
import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, Conv3D, ConvLSTM2D, concatenate, BatchNormalization,
    Dense, SeparableConv2D, Activation, Dropout, TimeDistributed
)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Visualization
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt

# Evaluation and time measurement
import time

# File handling and preprocessing
import os
import re
import glob
from scipy.ndimage import rotate
import random

# Type hinting
from typing import overload

In [ ]:
def set_global_determinism(seed=42):
    """
    Fix all seeds for Python, NumPy, and TensorFlow to ensure (as much as possible)
    reproducible results, including GPU determinism if possible.
    """
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_global_determinism(seed=42)

# **Data loader**

In [ ]:
def min_max_scaler(x: np.ndarray) -> np.ndarray:
    """
    Scale the input array x to the range [0, 1] using min-max normalization.

    Args:
        x (np.ndarray): Input array.

    Returns:
        np.ndarray: Scaled array in range [0, 1].
    """
    x_min = np.min(x)
    x_max = np.max(x)
    return (x - x_min) / (x_max - x_min + 1e-10)


"""
Custom Dataset Class for Spatio-Temporal City Growth Modeling
This class loads multi-year .tif files, constructs time-windowed samples,
performs optional augmentation (flip, rotation), and splits into patches if needed.
"""
class CityGrowthDataset:
    def __init__(self,
                 data_dir: str,
                 years: list,
                 channels: list,
                 window_size: int = 2,
                 patch_size: int = 20,
                 overlap: int = 10,
                 rotation_angle: int = 15,
                 do_flip: bool = True,
                 random_seed: int = 42):
        """
        Initialize the dataset with parameters.

        Args:
            data_dir (str): The directory path containing .tif files.
            years (list): A sorted list of years (e.g. [1980, 1990, 2000, 2010]).
            channels (list): List of channel names to load (e.g. ['lc1','lc2',...]).
            window_size (int): Number of consecutive years to use as input.
            patch_size (int): The size of patches to extract. If None or 0, no patch splitting.
            overlap (int): Overlap size between adjacent patches.
            rotation_angle (int): The step size (in degrees) for discrete rotation candidates.
                                  0 means no rotation.
            do_flip (bool): Whether to apply random flipping horizontally and vertically.
            random_seed (int): Seed for reproducibility in random operations.
        """
        self.data_dir = data_dir
        self.years = sorted(years)
        self.channels = channels
        self.window_size = window_size
        self.patch_size = patch_size
        self.overlap = overlap
        self.rotation_angle = rotation_angle
        self.do_flip = do_flip
        np.random.seed(random_seed)
        self.data_dict = self._load_all_data()
        self.samples = self._create_time_window_samples()

    def _load_all_data(self) -> dict:
        """
        Load all .tif files and stack channels for each year.
        Each channel is min-max scaled.

        Returns:
            dict: Mapping from each year to a numpy array (H, W, num_channels).
        """
        data_dict = {}
        all_tifs = glob.glob(os.path.join(self.data_dir, '*.tif'))

        year_pattern = re.compile(r'.*_(\d{4})_')
        file_map = {}
        for tif_path in all_tifs:
            base = os.path.basename(tif_path)
            match = year_pattern.match(base)
            if not match:
                continue
            year_str = match.group(1)
            year_int = int(year_str)

            ch_str = base.split('_')[-1].replace('.tif', '')
            file_map[(year_int, ch_str)] = tif_path


        for year in self.years:
            channel_arrays = []
            for ch in self.channels:
                if (year, ch) not in file_map:
                    raise ValueError(f"Missing file for year={year}, channel={ch}")
                tif_path = file_map[(year, ch)]
                with rasterio.open(tif_path) as src:
                    arr = src.read(1)
                    arr = arr.astype(np.float32)

                arr = min_max_scaler(arr)
                channel_arrays.append(arr)

            stacked = np.stack(channel_arrays, axis=-1)
            data_dict[year] = stacked

        return data_dict

    def _create_time_window_samples(self) -> list:
        samples = []
        for i in range(len(self.years) - self.window_size):
            x_years = self.years[i:i+self.window_size]
            y_year = self.years[i+self.window_size]

            X = np.stack([self.data_dict[y] for y in x_years], axis=0)

            lc1_idx = self.channels.index('lc1')
            Y = self.data_dict[y_year][..., lc1_idx:lc1_idx+1]

            samples.append((X, Y))
        return samples

    def __len__(self) -> int:
        return len(self.samples)

    def __getitem__(self, idx: int):
        """
        Returns a single sample (or multiple patches) at index idx.
        - Applies data augmentation (flip, rotation) to the entire sample.
        - Splits into patches if patch_size is specified.
        - Converts each patch to tf.Tensor.

        Returns:
            (list_of_X_tensors, list_of_Y_tensors):
                Each X_tensor has shape (window_size, patch_H, patch_W, C).
                Each Y_tensor has shape (patch_H, patch_W, 1).
        """
        X, Y = self.samples[idx]
        X_aug, Y_aug = self._augment(X, Y)
        X_patches, Y_patches = self._patch_split(X_aug, Y_aug)

        X_tensors = [tf.convert_to_tensor(xp, dtype=tf.float32) for xp in X_patches]
        Y_tensors = [tf.convert_to_tensor(yp, dtype=tf.float32) for yp in Y_patches]
        return X_tensors, Y_tensors

    def _augment(self, X: np.ndarray, Y: np.ndarray):
        """
        Apply data augmentation to input X and target Y.
        - Flip horizontally/vertically with 50% chance if do_flip=True.
        - Rotate by a discrete angle multiple if rotation_angle > 0.
        - Outside region after rotation is masked to 0.
        """
        if self.do_flip:
            if np.random.rand() < 0.5:
                X = np.flip(X, axis=1)
                Y = np.flip(Y, axis=0)
            if np.random.rand() < 0.5:
                X = np.flip(X, axis=2)
                Y = np.flip(Y, axis=1)


        if self.rotation_angle > 0:
            possible_angles = np.arange(0, 360, self.rotation_angle)
            angle = np.random.choice(possible_angles)

            rotated_X = []
            for t in range(X.shape[0]):
                x_t = X[t]
                mask = np.ones(x_t.shape[:2], dtype=np.float32)
                x_rot = rotate(x_t, angle=angle, axes=(0,1), reshape=False,
                               order=1, mode='constant', cval=0)
                m_rot = rotate(mask, angle=angle, axes=(0,1), reshape=False,
                               order=0, mode='constant', cval=0)
                x_rot = x_rot * m_rot[..., np.newaxis]  # mask out-of-bound
                rotated_X.append(x_rot)
            X = np.stack(rotated_X, axis=0)

            mask_y = np.ones(Y.shape[:2], dtype=np.float32)
            y_rot = rotate(Y, angle=angle, axes=(0,1), reshape=False,
                           order=1, mode='constant', cval=0)
            m_y_rot = rotate(mask_y, angle=angle, axes=(0,1), reshape=False,
                             order=0, mode='constant', cval=0)
            Y = y_rot * m_y_rot[..., np.newaxis]

        return X, Y

    def _patch_split(self, X: np.ndarray, Y: np.ndarray):
        """
        Split the input X and Y into patches of size patch_size x patch_size,
        with the specified overlap. If patch_size=None or 0, return the original image as is.
        If patch_size is None or 0, do not split.

        Args:
            X (np.ndarray): Input volume shape (time, H, W, C).
            Y (np.ndarray): Target shape (H, W, 1).

        Returns:
            (X_patches, Y_patches): Lists of np.ndarray patches.
                Each X patch: (time, patch_size, patch_size, C)
                Each Y patch: (patch_size, patch_size, 1)
        """
        if not self.patch_size:
            return [X], [Y]

        X_patches = []
        Y_patches = []
        _, H, W, _ = X.shape
        step = self.patch_size - self.overlap

        for i in range(0, H - self.patch_size + 1, step):
            for j in range(0, W - self.patch_size + 1, step):
                X_patch = X[:, i:i+self.patch_size, j:j+self.patch_size, :]
                Y_patch = Y[i:i+self.patch_size, j:j+self.patch_size, :]
                X_patches.append(X_patch)
                Y_patches.append(Y_patch)

        return X_patches, Y_patches

In [ ]:
data_directory = "your/data/path/Variables"
channels = ['lc1','lc2','lc3','lc4','lc6', 'lc7',
            'acc1','acc2','acc3','acc4','acc6','acc7',
            'accsws','accic','accwater']

patch_size = None
overlap = 0
rotation_angle = 90
do_flip = True

train_dataset = CityGrowthDataset(
    data_dir = data_directory,
    years     = [1980, 1990, 2000],
    channels  = channels,
    window_size     = 2,
    patch_size      = patch_size,
    overlap         = overlap,
    rotation_angle  = rotation_angle,
    do_flip         = do_flip,
    random_seed     = 42
)

test_dataset = CityGrowthDataset(
    data_dir = data_directory,
    years     = [1990, 2000, 2010],
    channels  = channels,
    window_size     = 2,
    patch_size      = patch_size,
    overlap         = overlap,
    rotation_angle  = 0,
    do_flip         = False,
    random_seed     = 42
)

X_patches, Y_patches = train_dataset[0]

print("Number of time-window samples in dataset:", len(train_dataset))
print("Number of patches in sample[0]:", len(X_patches))
print("X_patches[0] shape:", X_patches[0].shape, "Y_patches[0] shape:", Y_patches[0].shape)

In [ ]:
def train_generator():
    X_list, Y_list = train_dataset[0]
    for X_t, Y_t in zip(X_list, Y_list):
        yield X_t, Y_t

def test_generator():
    X_list, Y_list = test_dataset[0]
    for X_t, Y_t in zip(X_list, Y_list):
        yield X_t, Y_t

batch_size = 4

train_ds = tf.data.Dataset.from_generator(
    train_generator,
    output_signature=(
        tf.TensorSpec(shape=(2, patch_size, patch_size, 15), dtype=tf.float32),
        tf.TensorSpec(shape=(patch_size, patch_size, 1), dtype=tf.float32)
    )).batch(batch_size)

test_ds  = tf.data.Dataset.from_generator(
    test_generator,
    output_signature=(
        tf.TensorSpec(shape=(2, patch_size, patch_size, 15), dtype=tf.float32),
        tf.TensorSpec(shape=(patch_size, patch_size, 1), dtype=tf.float32)
    )).batch(batch_size)

In [ ]:
for step, (X_batch, Y_batch) in enumerate(train_ds):
    print("Train step:", step)
    print("X_batch.shape:", X_batch.shape)  # (batch_size, seq_len, patch_size, patch_size, channels)
    print("Y_batch.shape:", Y_batch.shape)  # (batch_size, patch_size, patch_size, 1)
    break

for step, (X_batch, Y_batch) in enumerate(test_ds):
    print("Test step:", step)
    print("X_batch.shape:", X_batch.shape)
    print("Y_batch.shape:", Y_batch.shape)
    break

In [ ]:
for X_patch, Y_patch in train_generator():
    print("X_patch.shape:", X_patch.shape, "Y_patch.shape:", Y_patch.shape)
    break

# **Network Model**

In [ ]:
def acc(y_true, y_pred):
    y_pred_bin = tf.cast(y_pred >= 0.5, tf.float32)
    return tf.reduce_mean(tf.cast(tf.equal(y_true, y_pred_bin), tf.float32))

def f1_score(y_true, y_pred):
    y_pred_bin = tf.cast(y_pred >= 0.5, tf.float32)
    tp = tf.reduce_sum(y_true * y_pred_bin)
    fp = tf.reduce_sum((1 - y_true) * y_pred_bin)
    fn = tf.reduce_sum(y_true * (1 - y_pred_bin))
    precision = tp / (tp + fp + 1e-7)
    recall = tp / (tp + fn + 1e-7)
    f1 = 2 * precision * recall / (precision + recall + 1e-7)
    return f1

## **Hyper-parameters**

In [ ]:
kn_s = 3
dsfilter1 = 8
dsfilter2 = 4
filter1 = 30
filter2 = 20
unit1 = 4

In [ ]:
seq_len = X_patch.shape[0]
input_row = X_patch.shape[1]
input_col = X_patch.shape[2]
input_chan = X_patch.shape[3]

seq_len, input_row, input_col, input_chan

## **Modeling**

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

class ReflectionPadding2DTime(Layer):
    """
    A custom layer to perform REFLECT padding on the spatial dimensions.
    Supports both 4D (batch, H, W, C) and 5D (batch, time, H, W, C) inputs.
    """
    def __init__(self, pad=(1,1), **kwargs):
        """
        Args:
            pad: tuple of (pad_height, pad_width)
        """
        super().__init__(**kwargs)
        self.pad = pad

    def call(self, x):
        ph, pw = self.pad
        input_rank = len(x.shape)

        if input_rank == 5:
            paddings = [[0,0], [0,0], [ph, ph], [pw, pw], [0,0]]
        elif input_rank == 4:
            paddings = [[0,0], [ph, ph], [pw, pw], [0,0]]
        else:
            raise ValueError("ReflectionPadding2DTime supports only 4D or 5D tensors.")

        return tf.pad(x, paddings=paddings, mode='REFLECT')

    def compute_output_shape(self, input_shape):
        ph, pw = self.pad
        input_shape = tf.TensorShape(input_shape).as_list()
        if len(input_shape) == 5:
            batch, time, h, w, c = input_shape
            h = h + 2*ph if h is not None else None
            w = w + 2*pw if w is not None else None
            return (batch, time, h, w, c)
        elif len(input_shape) == 4:
            batch, h, w, c = input_shape
            h = h + 2*ph if h is not None else None
            w = w + 2*pw if w is not None else None
            return (batch, h, w, c)
        else:
            raise ValueError("Input shape must be 4D or 5D.")

In [ ]:
with tf.device('/GPU:0'):

    inputs = Input(shape=(seq_len, input_row, input_col, input_chan), name='Input')
    DSC = ReflectionPadding2DTime(pad=(kn_s//2, kn_s//2))(inputs)
    DSC = TimeDistributed(SeparableConv2D(filters=dsfilter1, kernel_size=kn_s, padding='valid'))(DSC)
    DSC = TimeDistributed(BatchNormalization())(DSC)

    DSC = ReflectionPadding2DTime(pad=(kn_s//2, kn_s//2))(DSC)
    DSC = TimeDistributed(SeparableConv2D(filters=dsfilter2, kernel_size=kn_s, padding='valid'))(DSC)
    DSC = TimeDistributed(BatchNormalization())(DSC)

    DSC = ReflectionPadding2DTime(pad=(kn_s//2, kn_s//2))(DSC)
    DSC = TimeDistributed(Conv2D(1, kernel_size=kn_s, padding='valid', activation='sigmoid'))(DSC)

    DS_Conv_LSTM = ReflectionPadding2DTime(pad=(kn_s//2, kn_s//2))(DSC)
    DS_Conv_LSTM = ConvLSTM2D(filters=filter1, kernel_size=kn_s, padding='valid', return_sequences=True, activation='tanh')(DS_Conv_LSTM)
    DS_Conv_LSTM = BatchNormalization()(DS_Conv_LSTM)

    DS_Conv_LSTM = ReflectionPadding2DTime(pad=(kn_s//2, kn_s//2))(DS_Conv_LSTM)
    DS_Conv_LSTM = ConvLSTM2D(filters=filter2, kernel_size=kn_s, padding='valid', return_sequences=False, activation='tanh')(DS_Conv_LSTM)
    DS_Conv_LSTM = BatchNormalization()(DS_Conv_LSTM)

    DS_Conv_LSTM = TimeDistributed(Dense(units=unit1, activation='relu'))(DS_Conv_LSTM)

    DS_Conv_LSTM = ReflectionPadding2DTime(pad=(kn_s//2, kn_s//2))(DS_Conv_LSTM)
    Output = Conv2D(1, kernel_size=(kn_s, kn_s), padding='valid', activation='sigmoid')(DS_Conv_LSTM)

    DS_ConvLSTM_Model = Model(inputs=input, outputs = Output)
    DS_ConvLSTM_Model.compile(loss='mse', optimizer='adam', metrics=[acc, f1_score, 'mse'])

DS_ConvLSTM_Model.summary()

## **Model Train**

In [ ]:
with tf.device('/device:GPU:0'):
  start = time.time()
  epochs = 1000
  earlystop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
  )

  checkpoint_loss = ModelCheckpoint(
      filepath='your/data/path/Urban_Growth_Model_hanam/best_loss_ds_convlstm.h5',
      monitor='val_loss',
      save_best_only=True,
      mode='min',
      verbose=1
  )

  checkpoint_f1 = ModelCheckpoint(
      filepath='your/data/path/Urban_Growth_Model_hanam/best_f1_ds_convlstm.h5',
      monitor='val_f1_score',
      save_best_only=True,
      mode='max',
      verbose=1
  )

  hist = DS_ConvLSTM_Model.fit(
    train_ds,
    epochs=epochs,
    validation_data=test_ds,
    callbacks=[checkpoint_loss, checkpoint_f1, earlystop]
    )

  end = time.time()
  how_long = end - start

print(how_long // 3600, ' hours', (how_long % 3600) // 60, ' miniutes')

## **Result**

### *Graph*

In [ ]:
fig, loss_ax = plt.subplots()

loss_ax.plot(hist.history['loss'], 'r', label='train loss')
loss_ax.plot(hist.history['val_loss'],'y--', label='val loss')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')

loss_ax.legend(loc='upper right')

plt.show()

In [ ]:
fig, acc_ax = plt.subplots()

acc_ax.plot(hist.history['acc'], 'b', label='train acc')
acc_ax.plot(hist.history['val_acc'],'g--', label='val acc')

acc_ax.set_xlabel('epoch')
acc_ax.set_ylabel('accuracy')

acc_ax.legend(loc='lower right')

plt.show()

In [ ]:
fig, acc_ax = plt.subplots()

acc_ax.plot(hist.history['f1_score'], 'b', label='train f1_score')
acc_ax.plot(hist.history['val_f1_score'],'g--', label='val f1_score')

acc_ax.set_xlabel('epoch')
acc_ax.set_ylabel('accuracy')

acc_ax.legend(loc='lower right')

plt.show()

### *Map*

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.losses import MeanSquaredError

model_path = 'your/data/path/Urban_Growth_Model_hanam/best_f1_ds_convlstm.h5'
model = tf.keras.models.load_model(
    model_path,
    custom_objects={
        'acc': acc,
        'f1_score': f1_score,
        'mse': MeanSquaredError(),
        'ReflectionPadding2DTime': ReflectionPadding2DTime
    })

In [ ]:
X_list, Y_list = test_dataset [0]
print(f"Number of patches in test sample[0]: {len(X_list)}")

X_patch = X_list[0]
Y_patch = Y_list[0]

print("X_patch shape:", X_patch.shape)
print("Y_patch shape:", Y_patch.shape)

In [ ]:
X_input = tf.expand_dims(X_patch, axis=0)

pred = model.predict(X_input)
pred_map = pred[0]

print("pred_map shape:", pred_map.shape)

In [ ]:
im_1990 = X_patch[0, :, :, 0]
im_2000 = X_patch[1, :, :, 0]

pred_2010 = pred_map[..., 0]

gt_2010 = Y_patch[..., 0]

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 3))

plt.subplot(1, 4, 1)
plt.imshow(im_1990, cmap='gray')
plt.title("1990 lc1 (Input)")
plt.axis('off')

plt.subplot(1, 4, 2)
plt.imshow(im_2000, cmap='gray')
plt.title("2000 lc1 (Input)")
plt.axis('off')

plt.subplot(1, 4, 3)
plt.imshow(pred_2010, cmap='gray', vmin=0, vmax=1)
plt.title("Predicted Probability")
plt.axis('off')

plt.subplot(1, 4, 4)
plt.imshow(gt_2010, cmap='gray')
plt.title("2010 lc1 (Ground Truth)")
plt.axis('off')

plt.tight_layout()
plt.show()

# **Output to GIS**

In [ ]:
import os
import numpy as np
import rasterio
import tensorflow as tf

def min_max_scaler(x: np.ndarray) -> np.ndarray:
    x_min = np.min(x)
    x_max = np.max(x)
    return (x - x_min) / (x_max - x_min + 1e-10)


def load_year_stack(year, channels):
    channel_arrays = []
    for ch in channels:
        path = f"your/data/path/Urban_Growth_Model_hanam/Variables/hn_{year}_{ch}.tif"
        with rasterio.open(path) as src:
            arr = src.read(1).astype(np.float32)
        arr = min_max_scaler(arr)
        channel_arrays.append(arr)
    stacked = np.stack(channel_arrays, axis=-1)
    return stacked


def split_into_patches(X, patch_size=30, overlap=10):
    """
    Split input X (shape=(time, H, W, C)) into patches.
    Returns:
      - patches: list of patches of shape (time, patch_size, patch_size, C)
      - origins: list of (i, j) indicating the top-left coordinate of each patch in the original image.
    """
    t, H, W, C = X.shape
    step = patch_size - overlap
    patches = []
    origins = []
    for i in range(0, H - patch_size + 1, step):
        for j in range(0, W - patch_size + 1, step):
            patch = X[:, i:i+patch_size, j:j+patch_size, :]
            patches.append(patch)
            origins.append((i, j))
    return patches, origins

def predict_full_image(model, X_full, patch_size=30, overlap=10):
    """
    Given a full input image X_full of shape (time, H, W, C),
    predict the output map using a sliding window approach if patch_size is specified.
    Overlapping areas are averaged.

    Returns:
      pred_canvas: predicted map of shape (H, W)
    """
    t, H, W, C = X_full.shape
    if patch_size is None or patch_size == 0:
        X_input = np.expand_dims(X_full, axis=0)
        pred = model.predict(X_input)
        pred_map = np.squeeze(pred, axis=(0, -1))
        return pred_map

    patches, origins = split_into_patches(X_full, patch_size, overlap)
    pred_canvas = np.zeros((H, W), dtype=np.float32)
    count_canvas = np.zeros((H, W), dtype=np.float32)

    for patch, (i, j) in zip(patches, origins):
        patch_input = np.expand_dims(patch, axis=0)
        patch_pred = model.predict(patch_input)
        patch_pred_map = np.squeeze(patch_pred, axis=(0, -1))
        pred_canvas[i:i+patch_size, j:j+patch_size] += patch_pred_map
        count_canvas[i:i+patch_size, j:j+patch_size] += 1.0

    pred_canvas[count_canvas > 0] /= count_canvas[count_canvas > 0]
    return pred_canvas


def unified_predict(model, X_full, patch_size, overlap):
    """
    Predict full image map from input X_full (shape=(time, H, W, C)) using either
    full image prediction (if patch_size is None or 0) or patch-based prediction.
    """
    return predict_full_image(model, X_full, patch_size, overlap)


def save_predicted_map(pred_map, reference_tif, output_tif):
    with rasterio.open(reference_tif) as src:
        profile = src.profile.copy()
    profile.update({'count': 1, 'dtype': 'float32'})
    with rasterio.open(output_tif, 'w', **profile) as dst:
        dst.write(pred_map.astype(np.float32), 1)
    print(f"Saved predicted map to {output_tif}")

In [ ]:
reference_tif = "your/data/path/Urban_Growth_Model_hanam/Variables/hn_1980_lc1.tif"
output2010_tif = "your/data/path/Urban_Growth_Model_hanam/Pred_10s_DSConvLSTM.tif"
output2020_tif = "your/data/path/Urban_Growth_Model_hanam/Pred_20s_DSConvLSTM.tif"

channels = ['lc1','lc2','lc3','lc4','lc6','lc7',
            'acc1','acc2','acc3','acc4','acc6','acc7',
            'accsws','accic','accwater']

X_1990 = load_year_stack(1990, channels)
X_2000 = load_year_stack(2000, channels)
X_2010 = load_year_stack(2010, channels)

X_full_2010 = np.stack([X_1990, X_2000], axis=0)
X_full_2020 = np.stack([X_2000, X_2010], axis=0)

pred_map_2010 = unified_predict(model, X_full_2010, patch_size, overlap)
pred_map_2020 = unified_predict(model, X_full_2020, patch_size, overlap)

save_predicted_map(pred_map_2010, reference_tif, output2010_tif)
save_predicted_map(pred_map_2020, reference_tif, output2020_tif)